Important information!
/gscratch/pfaendtner/SUM2018/Wes

<font color='red'>6-28-2018</font>

I found out why the ffmaker script isn't working.
1. First off, I was suspicious that the problem was Gaussian. I couldn't tell at first, because I wasn't getting any output files or error files at all (although they should be combined a indicated by the options). I knew it was Gaussian because the file ran extrememly fast and the first step that should take any time was running the Gaussian simulation.
2. Turns out, the -j oe option doesn't seem to be working for me. If I use this option I don't get any files, output or otherwise. The combined option never appears. I had to switch to making separate error and output files
3. Once I finally got the output and error files working, I was finally able to get proper error messages from BMI_TF2.bash. That took way longer than it needed to. Turns out, the problem is Gaussian! AGAIN! The exact problem was the the command "g09 is not recognized".

Trying to find a solution to running Gaussian.
1. First two attempts were to export the running of Gaussian outside of the .bash script by sourcing a Gaussian submission script that I made a couple years ago. It is called gaussian_auto.pbs. This failed. I got an error that my "user ID is bad". I'm not sure what this means exactly, but I can run the auto submission script if I got straight from the command line.
2. The third solution worked! Instead of trying to export to another file, I just copied everything that was in gaussian_auto.pbs, except for the options at the very top. I was entirely confident that this would work because I thought those commands needed to be in the .pbs, and adding a degree of separation would cause issues. Apparently not!

Notes about running ffmaker.bash (also known as BMI_TF2.bash)
This script can be really annoying to run on its own because it automatically renames files. For example if you already have a prepared input file "BMI.com" then the beginning of the script will automatically rename it to "BMI.backup.com" because it thinks it is a file from a previous simulation. So the backups aren't actual copies of whats running, but that of an old simluation. You can manually change the script to run "BMI.backup.com", but then it gets upset later on because it can't find "BMI.log". It seems there are other locations which would need to be changed. 

Easy solution?
Just go to line 132 and comment out the next three lines. This takes care of everything. 

I was working inside the "structures" directory, so now I have to go back to the "inputs" directory and make some overall changes. Notes:
1. I specified the working directory in boxmaker.pbs (also known as BMI_TF2.pbs) manually. I'm not sure if this also needs to be done with GROMACS.pbs. Either way, I should make this into a variable eventually.
2. Added in extra lines to BMI_TF2.bash to make Gaussian work (see above).
3. I deleted some old lines that I used for echo testing.
4. Splitting up all the error and log files, for now at least. If I can figure out why they wont' appear when combined and I can fix that, I'll put them back together. I've defined the names of the files manually, but I need to make them into variables so they will differentiate in the future when I auto run a bunch of simulations together. 

Note: Pulling and pushing files from HYAK.
<font color="blue">Into HYAK (from local terminal)</font>
scp filename user@ikt.hyak.uw.edu:/path
<font color="blue">Out of HYAK</font>
scp user@hyak.washington.edu:/path .

I'm having issues with waiting in the queue for a really long time. Maybe I could try shorting my walltime a little bit? I don't think I need as much as is listed. 

Things I need to do:
1. Set up GitHub branches for Wes and Luke's projects. Let them know where they are. Push the Jupyter notebooks as well.
2. I need to figure out some funnieness with the output and error file options. First off, the -j oe option only sometimes works. In the case of gaussian_auto.pbs it works and forms the file immediately so that I can tail it. On the other hand, only -e and -o work for the .bash file. The files also don't form immediately so I can't tail them to watch simulation progress. I'll run some tests in the tests directory. 
3. Start thinking about the auto-submission script. I need to be very careful that all the log files are stored before they are written over and that everything is stored in properly labeled directories.
4. I *really* need to make some sort of automatic prompt for checking that the directory has the right roots. This keeps screwing me over...

In [ ]:
# Code which I tried to use to export running of Gaussian.

# Attempt 1, FAILED
# Stuff, stuff, .bash stuff
source gaussian_auto.pbs

# Attempt 2, FAILED
# Stuff, stuff, .bash stuff
qsub gaussian_auto.pbs

# The error
# qsub: submit error (Bad UID for job execution MSG=ruserok failed validating maneko/maneko from n0723) tail: cannot open `BMI.log' for reading: No such file or directory

# Attempt 3, SUCCESS
export g09root=/gscratch/pfaendtner/codes/G09
export GAUSS_SCRDIR=/scr
. $g09root/g09/bsd/g09.profile
g09 $RESIDUE_NAME.com

Questions for Wes
1. Any idea why the heck the g09 command wasn't recognized?? I'm assuming it worked for him and he didn't have to change anything since I copied these scripts straight from his directories.
2. Is there an easy, automatic way to qsub jobs with a script? I found some pretty complicated solutions online involving Python, but I'm thinking there has to be an easier way to do this. Otherwise, I could have problems in the future when I'm trying to work with all 82 salts. I just want to submit one job and run all of them.

<font color='red'>6-29-18</font>

The simulation runs sucessfully through the creation of the cation and anion force fields. ie. It passes through spFF.sh sucessfully. There is a problem in spMD.sh with a filepath, I'm not sure how to fix it! The error is that pacmol cannot be found along the specified filepath. Hopefully Wes or Luke know. If not, I can ask Jim directly. 

Update: We got the problem fixed. It was super simple and I just had to import in a filepath as provided by Wes. Just to be sure I put it in the terminal and in the scripts.

I've begun writing the script to automate things. I'll start out with a small test with just a single system and follow step-by-step especially making sure that all the error files get named properly and such.

<font color='red'>7-2-18</font>

Good news! I got the BMI_TF2 system working completely. There are just a couple random places where the scripts seem to get stuck in the wrong directory or have trouble finding certain files. If I go in an manually specify some filepaths, then things run okay and it doesn't take too much longer since the previous files don't need to be rerun. This will just become a bit of an issue when I try to automate things. 

Currrently, I running and waiting on a C00_A00 system.

Checklist of things that the auto-script needs to be able to do. 
1. Confirm root directories and such, ONCE. 
2. The script needs to be able to just start with the .pdb files as provided by Wes' python script.
3. Run "source salt.sh salt.inp". This means being able to go in an change the name of the cation and anion in the salt.inp file. Also, making the guess based on what's avaliable. For the first runs I may just standardize at 1.00 and later pull data for 80%. 90%?
4. "source master.sh CAT_AN" to run the scripts.
5. Once the scripts are running be able to name the separate error files in the structures directory. Once the systems directory is entered, it should be fine since a separate directory is made. 
6. Send out an email if a certain salt doesn't converge, and be able to continue one with the next salt. Such as "if a certain salt takes more than 5 hours to run, make a note (via email) and then move onto the next salt). 
7. I would really like it if the script could spit out an output which tells me how long each step of the simulation took?

Quesetions for Wes:
1. Is it worth comparing results for different standardized guesses? (Like everything 80% vs. 40%).
    Not really. Better to stay close to within 10% of the real value.
2. Is it worth rerunning the same system again? Will any random variables actually influence anything strongly enough to make a change worth noting?
    Definitely!
3. Is it worth recording the amount of time which each simulation took? Or even each step?
    Not really, but it's worth knowing so that you can get a feel for your workflow.
4. How many nodes can I use? If I run this script, I'm planning on staggering it so that maybe 2-3 salts are running at once. This would be 2-3 nodes at any given time. Which types of nodes can I use?
    Could probably use up to five. 
    
Bad news: Comparing with Wes, I got a really low value for the heat capacity vs. the density which was actually really close. I am going to rerun the system at Wes's request. 

I ended up removing the line in the .pbs file where I specify directory movement. That was only really necessary while I was testing and didn't have earlier scripts to direct the job submission file. It was just really messing with things.

Things that I need to do:
1. I'd liket to make it so that the job submission names change with each salt. It's not going to be helpful unless I'm actually watching qstat - but it might have some utility and shouldn't require too much to implement.

<font color='red'>7-3-18</font>

I reran the BMI_TF2 system and... the heat capacity is still super low compared to what Wes got. I got around 500 while Wes gets around 1500. His should be correct.

Notes about errors:
At the very last step, the simluation gets stuck. It should be in the correct directory, but can't find a certain file for some reason. I have to manually insert a director